<a href="https://colab.research.google.com/github/NitinVerma2027/PRML-Apr2025/blob/main/knn_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import zipfile
import os

In [8]:
# Define the path to the uploaded file
zip_path = "/content/MNIST_CSV.zip"
extract_path = "/mnt/data/mnist_data"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the extracted files
extracted_files = os.listdir(extract_path)
print("Extracted Files:", extracted_files)

Extracted Files: ['readme.md', 'generate_mnist_csv.py', 'mnist_train.csv', 'mnist_test.csv']


In [9]:
# Load the CSV files (assuming standard MNIST naming conventions)
train_file = os.path.join(extract_path, "mnist_train.csv")  # Update filename if needed

# Read the CSV files
train_data = pd.read_csv(train_file)

# Display first few rows of each dataset
print("\nTraining Data Preview:")
print(train_data.head())

# Check dataset shape
print("\nTraining Data Shape:", train_data.shape)


Training Data Preview:
   5  0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  ...  0.608  0.609  0.610  \
0  0  0    0    0    0    0    0    0    0    0  ...      0      0      0   
1  4  0    0    0    0    0    0    0    0    0  ...      0      0      0   
2  1  0    0    0    0    0    0    0    0    0  ...      0      0      0   
3  9  0    0    0    0    0    0    0    0    0  ...      0      0      0   
4  2  0    0    0    0    0    0    0    0    0  ...      0      0      0   

   0.611  0.612  0.613  0.614  0.615  0.616  0.617  
0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0  

[5 rows x 785 columns]

Training Data Shape: (59999, 785)


In [11]:
# Load the CSV files (assuming standard MNIST naming conventions)
test_file = os.path.join(extract_path, "mnist_test.csv")  # Update filename if needed

# Read the CSV files
test_data = pd.read_csv(test_file)

# Display first few rows of each dataset
print("\nTesting Data Preview:")
print(test_data.head())

# Check dataset shape
print("Testing Data Shape:", test_data.shape)


Testing Data Preview:
   7  0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  ...  0.658  0.659  0.660  \
0  2  0    0    0    0    0    0    0    0    0  ...      0      0      0   
1  1  0    0    0    0    0    0    0    0    0  ...      0      0      0   
2  0  0    0    0    0    0    0    0    0    0  ...      0      0      0   
3  4  0    0    0    0    0    0    0    0    0  ...      0      0      0   
4  1  0    0    0    0    0    0    0    0    0  ...      0      0      0   

   0.661  0.662  0.663  0.664  0.665  0.666  0.667  
0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0  

[5 rows x 785 columns]
Testing Data Shape: (9999, 785)


In [12]:
import numpy as np
import pandas as pd
from collections import Counter


In [13]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))


In [14]:
def knn_predict(X_train, y_train, X_test, k=3):
    y_pred = []
    for test_sample in X_test:
        distances = [euclidean_distance(test_sample, train_sample) for train_sample in X_train]
        k_indices = np.argsort(distances)[:k]
        k_nearest_labels = [y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)[0][0]
        y_pred.append(most_common)
    return np.array(y_pred)

In [15]:
# Load MNIST CSV files, use the extract path used previously
extract_path = "/mnt/data/mnist_data" # use the same extract path as before
train_data = pd.read_csv(os.path.join(extract_path, "mnist_train.csv"))  # Provide the full path
test_data = pd.read_csv(os.path.join(extract_path, "mnist_test.csv"))  # Provide the full path


# Extract features and labels
X_train = train_data.iloc[:, 1:].values / 255.0  # Normalize pixel values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values / 255.0
y_test = test_data.iloc[:, 0].values

# Perform k-NN classification
y_pred = knn_predict(X_train, y_train, X_test, k=3)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"\n✅ k-NN Accuracy (Manual): {accuracy * 100:.2f}%")



✅ k-NN Accuracy (Manual): 97.17%


In [16]:
# Generate classification report
def classification_report_manual(y_true, y_pred):
    unique_labels = np.unique(y_true)
    report = ""
    for label in unique_labels:
        tp = np.sum((y_true == label) & (y_pred == label))
        fp = np.sum((y_true != label) & (y_pred == label))
        fn = np.sum((y_true == label) & (y_pred != label))
        tn = np.sum((y_true != label) & (y_pred != label))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        report += f"Class {label}: Precision: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}\n"
    return report

print("\nClassification Report:")
print(classification_report_manual(y_test, y_pred))


Classification Report:
Class 0: Precision: 0.97, Recall: 0.99, F1-score: 0.98
Class 1: Precision: 0.97, Recall: 1.00, F1-score: 0.98
Class 2: Precision: 0.98, Recall: 0.97, F1-score: 0.98
Class 3: Precision: 0.97, Recall: 0.97, F1-score: 0.97
Class 4: Precision: 0.98, Recall: 0.97, F1-score: 0.97
Class 5: Precision: 0.96, Recall: 0.96, F1-score: 0.96
Class 6: Precision: 0.98, Recall: 0.99, F1-score: 0.98
Class 7: Precision: 0.96, Recall: 0.97, F1-score: 0.96
Class 8: Precision: 0.99, Recall: 0.95, F1-score: 0.97
Class 9: Precision: 0.96, Recall: 0.96, F1-score: 0.96

